In [ ]:
!pip3 install cupy

     |████████████████████████████████| 1.6 MB 5.2 MB/s 
ERROR: Operation cancelled by user


In [28]:
import pandas as pd
import numpy as np
import torch
import json
import itertools


def load_recommender_data(path_in_str):
    json_file = open(path_in_str)
    data = json.load(json_file)
    df = pd.DataFrame.from_dict(data, orient='index')
    df = df[df['error'].isna()]

    # %%
    df['user'] = df.index
    df.reset_index(inplace=True)
    df['user_id'] = df.index

    # %%
    def melt_series(s):
        lengths = s.str.len().values
        flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
        idx = np.repeat(s.index.values, lengths)
        return pd.Series(flat, idx, name=s.name)


    user_df = melt_series(df.data).to_frame().join(df.drop('data', 1))

    # %%
    user_df['anime_id'] = user_df['data'].apply(lambda x : x['node']['id'])
    user_df['status'] = user_df['data'].apply(lambda x : x['list_status']['status'])
    user_df['score'] = user_df['data'].apply(lambda x : x['list_status']['score'])
    user_df['is_rewatching'] = user_df['data'].apply(lambda x : x['list_status']['is_rewatching'])


    # %%
    user_df = user_df[['user', 'user_id', 'anime_id', 'status', 'score', 'is_rewatching']]
    user_df = user_df.rename({'score': 'user_score'}, axis=1)

    # %%
    path_in_str = '/content/data/anime_list_final_231.json'
    json_file = open(path_in_str)
    data = json.load(json_file)
    anime_df_raw = pd.DataFrame.from_dict(data, orient='index')
    anime_df_raw = anime_df_raw[anime_df_raw['error'] != 'not_found']
    anime_df = anime_df_raw[['id', 'title', 'mean', 'genres', 'statistics', 'num_episodes']]
    anime_df = anime_df.dropna()

    # %%
    anime_df['genres_name'] = anime_df['genres'].apply(lambda x : [a['name'] for a in x])
    anime_df['genres_id'] = anime_df['genres'].apply(lambda x : [a['id'] for a in x])

    # %%
    anime_df['watching'] = anime_df['statistics'].apply(lambda x : x['status']['watching'])
    anime_df['num_list_users'] = anime_df['statistics'].apply(lambda x : x['num_list_users'])
    anime_df['completed'] = anime_df['statistics'].apply(lambda x : x['status']['completed'])
    anime_df['plan_to_watch'] = anime_df['statistics'].apply(lambda x : x['status']['plan_to_watch'])
    anime_df['dropped'] = anime_df['statistics'].apply(lambda x : x['status']['dropped'])
    anime_df['on_hold'] = anime_df['statistics'].apply(lambda x : x['status']['on_hold'])

    # %%
    anime_df.drop(['genres', 'statistics'], axis=1, inplace=True)

    # %%
    anime_df.rename({'id': 'anime_id'}, axis=1, inplace=True)



    # %%
    df_merge_raw = pd.merge(anime_df, user_df, on = 'anime_id')

    # %%
    df_merge = df_merge_raw[['anime_id', 'title', 'genres_name', 'num_episodes', 'mean', 'num_list_users', 'user_id', 'user_score']]

    # %%
    df_merge.rename({'title': 'name', 'genres_name': 'genre', 'num_episodes': 'episodes', 'mean': 'rating_x', 'num_list_users': 'members', 'user_score': 'rating_y'}, axis=1, inplace= True)

    # %%
    df_merge['rating_x'] = df_merge['rating_x'].astype(int)
    df_merge['rating_x'] = df_merge['rating_x'].round()

    return df_merge, anime_df, user_df


def main():
    path_in_str = '/content/data/user_data.json'
    load_recommender_data(path_in_str)


In [29]:
df_merge, anime_df, user_df =  load_recommender_data('data/user_data.json')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [30]:
new_indices = range(len(anime_df.index))
dict_ids = dict(zip(list(anime_df['anime_id']), new_indices))
dict_ids

{1.0: 0,
 6.0: 1,
 8.0: 2,
 16.0: 3,
 18.0: 4,
 20.0: 5,
 22.0: 6,
 24.0: 7,
 26.0: 8,
 28.0: 9,
 30.0: 10,
 32.0: 11,
 43.0: 12,
 45.0: 13,
 47.0: 14,
 49.0: 15,
 51.0: 16,
 53.0: 17,
 55.0: 18,
 57.0: 19,
 59.0: 20,
 61.0: 21,
 63.0: 22,
 65.0: 23,
 67.0: 24,
 69.0: 25,
 72.0: 26,
 74.0: 27,
 76.0: 28,
 79.0: 29,
 81.0: 30,
 83.0: 31,
 85.0: 32,
 87.0: 33,
 89.0: 34,
 91.0: 35,
 93.0: 36,
 95.0: 37,
 97.0: 38,
 99.0: 39,
 101.0: 40,
 103.0: 41,
 105.0: 42,
 107.0: 43,
 109.0: 44,
 111.0: 45,
 113.0: 46,
 115.0: 47,
 117.0: 48,
 119.0: 49,
 121.0: 50,
 123.0: 51,
 125.0: 52,
 127.0: 53,
 129.0: 54,
 131.0: 55,
 133.0: 56,
 135.0: 57,
 137.0: 58,
 139.0: 59,
 142.0: 60,
 144.0: 61,
 146.0: 62,
 148.0: 63,
 150.0: 64,
 152.0: 65,
 154.0: 66,
 156.0: 67,
 158.0: 68,
 160.0: 69,
 162.0: 70,
 164.0: 71,
 166.0: 72,
 168.0: 73,
 170.0: 74,
 173.0: 75,
 175.0: 76,
 177.0: 77,
 179.0: 78,
 181.0: 79,
 183.0: 80,
 185.0: 81,
 187.0: 82,
 189.0: 83,
 191.0: 84,
 193.0: 85,
 195.0: 86,
 197.0: 8

In [31]:
anime_df.replace({'anime_id': dict_ids}, inplace=True)
user_df.replace({'anime_id': dict_ids}, inplace=True)
df_merge.replace({'anime_id': dict_ids}, inplace=True)

In [32]:
df_merge.head()
sample = df_merge.sample(n=10000)
sample.shape

(10000, 8)

In [33]:
df_merge.dtypes

anime_id    float64
name         object
genre        object
episodes    float64
rating_x      int64
members       int64
user_id       int64
rating_y      int64
dtype: object

In [34]:
df_merge['anime_id'] = df_merge['anime_id'].astype(np.int32)
df_merge['rating_x'] = df_merge['rating_x'].astype(np.int32)
df_merge['user_id'] = df_merge['user_id'].astype(np.int32)
df_merge = df_merge[['anime_id', 'rating_x','user_id']]
df_merge.dtypes

anime_id    int32
rating_x    int32
user_id     int32
dtype: object

In [35]:
df_merge = df_merge.sample(n=5)
df_merge

,anime_id,rating_x,user_id
31006,9601,7,7
7684,3083,7,7
19452,5701,6,80
9033,3357,6,32
25719,6972,7,35


In [36]:
# rating matrix with rows as animes and columns as users
matrix_rating_user = np.ndarray(
    shape=(np.max(df_merge.anime_id.values), np.max(df_merge.user_id.values)+1),
    dtype=np.uint8)
matrix_rating_user

array([[243, 181,  22, ...,   0,   0, 243],
       [181,  22,   0, ...,   0, 243, 181],
       [ 22,   0,   0, ..., 243, 181,  22],
       ...,
       [  0,   0,   9, ...,   0,   0,   0],
       [  0,   5,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   7]], dtype=uint8)

In [37]:
matrix_rating_user[df_merge.anime_id.values-1, df_merge.user_id.values] = df_merge.rating_x.values
matrix_rating_user

array([[243, 181,  22, ...,   0,   0, 243],
       [181,  22,   0, ...,   0, 243, 181],
       [ 22,   0,   0, ..., 243, 181,  22],
       ...,
       [  0,   0,   9, ...,   0,   0,   0],
       [  0,   5,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   7]], dtype=uint8)

In [38]:
matrix_norm = matrix_rating_user - np.asarray([(np.mean(matrix_rating_user, 1))]).T
matrix_norm

array([[184.9382716 , 122.9382716 , -36.0617284 , ..., -58.0617284 ,
        -58.0617284 , 184.9382716 ],
       [123.7037037 , -35.2962963 , -57.2962963 , ..., -57.2962963 ,
        185.7037037 , 123.7037037 ],
       [-33.33333333, -55.33333333, -55.33333333, ..., 187.66666667,
        125.66666667, -33.33333333],
       ...,
       [ -0.71604938,  -0.71604938,   8.28395062, ...,  -0.71604938,
         -0.71604938,  -0.71604938],
       [ -1.        ,   4.        ,  -1.        , ...,  -1.        ,
         -1.        ,  -1.        ],
       [ -0.79012346,  -0.79012346,  -0.79012346, ...,  -0.79012346,
         -0.79012346,   6.20987654]])

In [39]:
# Compute the Singular Value Decomposition (SVD).
cuda  = torch.device('cuda')
A = matrix_norm.T / np.sqrt(matrix_rating_user.shape[0] - 1)
t = torch.from_numpy(A).to(device=cuda)     # converted to tensor
U, S, Vt = torch.svd(t)
V = Vt.cpu().detach().numpy()   #convert Vt to numpy array

In [40]:
V

array([[-2.99658270e-02,  1.37740953e-02,  2.21613032e-02, ...,
         2.39552819e-04, -4.54425677e-04, -3.17034287e-01],
       [-3.09450330e-02, -9.93941124e-03,  1.94686872e-02, ...,
         9.42326548e-04, -2.90224102e-05, -4.86610849e-01],
       [-1.34668651e-02, -2.85117316e-02, -2.24803397e-02, ...,
        -5.36450009e-04,  8.98563857e-06,  8.44949221e-02],
       ...,
       [ 4.23316789e-05,  4.76405876e-04, -5.86393526e-04, ...,
        -2.26780937e-04, -1.44796987e-05,  5.35933274e-05],
       [-3.97042715e-04,  4.76043943e-04, -7.91432608e-05, ...,
         2.48274114e-05, -5.94441986e-06,  6.22368699e-05],
       [-4.96619003e-04, -6.40623837e-06,  5.61324345e-04, ...,
        -3.72752392e-04,  5.41499999e-04, -2.51505394e-04]])

In [44]:
def cosine_similarity_sort(r_data, anime_id, top_n=10):
    ind = anime_id - 1 
    anime_row = r_data[ind, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', r_data, r_data))
    matrix_similarity = np.dot(anime_row, r_data.T) / (magnitude[ind] * magnitude)
    sorted_indices = np.argsort(-matrix_similarity)
    return sorted_indices[:top_n]

In [45]:
def get_most_similar_anime(anime_df, anime_id, index_list):
    print('Best recommendations for {0}: \n'.format(
    anime_df[anime_df.anime_id == anime_id].title.values[0]))
    for id in index_list + 1:
        print(anime_df[anime_df.anime_id == id].title.values[0])

In [46]:
k = 50      #k-principal components to represent anime, anime_id to find recommendations
anime_id = 2        # id for which we want recommendation
top_n = 10       
rep_data = V.T[:, :k] # representative data
index_list = cosine_similarity_sort(rep_data, anime_id, top_n)

#Get the top N recommendations
get_most_similar_anime(anime_df, anime_id, index_list)

Best recommendations for Bouken Ou Beet: 

Bouken Ou Beet
Kamikaze Kaitou Jeanne
Ou Dorobou Jing
Mobile Suit Victory Gundam
Green Green
Mahou Shoujo Lyrical Nanoha
Neon Genesis Evangelion: The End of Evangelion
Cluster Edge
Corrector Yui
El Hazard: The Magnificent World
